In [64]:
import pandas as pd 
import datetime as dt
import numpy as np
import keras
import plotly.graph_objects as go

from keras.models import Sequential
import tensorflow as tf
from alphaRNN import *
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.regularizers import l1
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
data = pd.read_csv("hourlySummaries.csv")
data = data.set_index("hour")
data = data[data.columns[1:]]
# data = data.iloc[:1000,]

train_weight = 0.80
split = int(len(data)*train_weight)
df_train = data.iloc[:split].copy()
df_test = data.iloc[split:].copy()

In [74]:
X_train = df_train.iloc[:,list(range(1,32))]
X_test = df_test.iloc[:,list(range(1,32))]
y_train = df_train["aaveHourlyChange"]
y_test = df_test["aaveHourlyChange"]

In [75]:
x_train_reg = np.array(X_train)
x_test_reg = np.array(X_test)
y_train_reg = np.array(y_train)
y_test_reg = np.array(y_test)

In [76]:
x_train_reg = np.reshape(x_train_reg, (x_train_reg.shape[0], x_train_reg.shape[1], 1))
x_test_reg = np.reshape(x_test_reg, (x_test_reg.shape[0], x_test_reg.shape[1], 1))

In [77]:
keras.backend.clear_session()

In [88]:
model = keras.Sequential()
model.add(keras.layers.LSTM(50, activation='relu', input_shape=(x_train_reg.shape[1], 1)))
model.add(keras.layers.Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(x_train_reg, y_train_reg, epochs=10, validation_split=0.2, verbose=1)

y_pred = model.predict(x_test_reg)

Train on 6944 samples, validate on 1736 samples
Epoch 1/10


KeyboardInterrupt: 

# Alpha RNN

In [81]:
def AlphaRNN_(n_units = 10, l1_reg=0, seed=0):
  model = Sequential()
  model.add(AlphaRNN(n_units, activation='tanh', kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed), recurrent_initializer=keras.initializers.orthogonal(seed), kernel_regularizer=l1(l1_reg), input_shape=(x_train_reg.shape[1], x_train_reg.shape[-1]), unroll=True))
  model.add(Dense(1, kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed), kernel_regularizer=l1(l1_reg)))
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

In [82]:
max_epoches=2000
batch_size=1000

es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=100, min_delta=1e-7, restore_best_weights=True)

# params = {'alpharnn': {'model':'', 'function':AlphaRNN_,'l1_reg':0.0, 'H':10,'color': 'green', 'label': '$\\alpha$-RNN' }}

In [83]:
model = AlphaRNN_(10,0.0)
model.fit(x_train_reg,y_train_reg,epochs=10, batch_size=batch_size,callbacks=[es],shuffle=False)


Epoch 1/10
8680/8680 [==============================] - 4s 435us/step - loss: 30.5015
Epoch 2/10
8680/8680 [==============================] - 1s 61us/step - loss: 30.0755
Epoch 3/10
8680/8680 [==============================] - 1s 69us/step - loss: 29.6712
Epoch 4/10
8680/8680 [==============================] - 1s 66us/step - loss: 29.2888
Epoch 5/10
8680/8680 [==============================] - 1s 65us/step - loss: 28.9286
Epoch 6/10
8680/8680 [==============================] - 1s 63us/step - loss: 28.5881
Epoch 7/10
8680/8680 [==============================] - 1s 67us/step - loss: 28.2630
Epoch 8/10
8680/8680 [==============================] - 1s 65us/step - loss: 27.9480
Epoch 9/10
8680/8680 [==============================] - 1s 64us/step - loss: 27.6488
Epoch 10/10
8680/8680 [==============================] - 1s 64us/step - loss: 27.3868


In [86]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
alpha_rnn_1 (AlphaRNN)       (None, 10)                121       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 132
Trainable params: 132
Non-trainable params: 0
_________________________________________________________________


In [87]:
y_pred_train = model.predict(x_train_reg, verbose = 1)
MSE_train= mean_squared_error(y_train,y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

y_pred = model.predict(x_test_reg, verbose = 1)
MSE_test= mean_squared_error(y_test,y_pred)
r2_test = r2_score(y_test, y_pred)




2171/2171 [==============================] - 0s 78us/step
